In [ ]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In this problem you, can simulate a simplified airport security system at a busy airport. Passengers arrive
according to a Poisson distribution with $\lambda$$_{1}$ = 5 per minute (i.e., mean interarrival rate $\mu$$_{1}$ = 0.2 minutes)
to the ID/boarding-pass check queue, where there are several servers who each have exponential
service time with mean rate $\lambda$$_{2}$ = 0.75 minutes. [Hint: model them as one block that has more than one
resource.] After that, the passengers are assigned to the shortest of the several personal-check queues,
where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1
minute).
Use the Arena software (PC users) or Python with SimPy (PC or Mac users) to build a simulation of the
system, and then vary the number of ID/boarding-pass checkers and personal-check queues to
determine how many are needed to keep average wait times below 15 minutes. [If you’re using SimPy,
or if you have access to a non-student version of Arena, you can use λ1 = 50 to simulate a busier airport.]

### DISCLAIMER - THIS PDF DOCUMENT IS LONG BECAUSE I'M OUTPUTTING THE STATUS OF EVERY CUSTOMER FOR A 24 HOUR PERIOD AT THE AIRPORTS. THIS OBVIOUSLY PRINTS OUT A LOT OF DATA DURING SIMULATION

In [ ]:
def airport_model(num_servers, num_scanners, seed, lambda_):
    
    #set passengers arriving according to poisson
    def generate_interarrival():
        return np.random.poisson(1/lambda_)
    
    #set servers effeciency to exponential distribution
    def generate_service():
        return np.random.exponential(0.75)
    
    #set personal scanning to uniform distribution
    def generate_screening():
        return np.random.uniform(0.5, 1.0)

    #main simulation execution function
    def customer_run(env, servers, screening):
        i = 0
        while True:
            i += 1
            yield env.timeout(generate_interarrival())
            
            env.process(customer(env, i, servers, screening))
            
    #secondary simulation execution function       
    def customer(env, customer, servers, screening):
        with servers.request() as request:
            with screening.request() as srequest:
                tick = env.now
                print(env.now, 'minutes: customer {} arrives'.format(customer))
                yield request
                print(env.now, 'minutes: customer {} is being served'.format(customer))
                yield env.timeout(generate_service())
                print(env.now, 'minutes: customer {} to screening'.format(customer))
                yield srequest
                yield env.timeout(generate_screening())
                print(env.now, 'minutes: customer {} through security'.format(customer))
                tock = env.now
                times.append((tock - tick))
                print('TOTAL WAIT FOR CUSTOMER {}: {} MINS'.format(customer, round((tock - tick),2)))
            
    #track queue times and lengths       
    def observe(env, servers, screening):
        while True:
            obs_times.append(env.now)
            seq_length.append(len(servers.queue))
            scq_length.append(len(screening.queue))
            yield env.timeout(1)
        
        
    #instantiate simulation environment    
    env = simpy.Environment()
    
    #amount of servers to adjust
    servers = simpy.Resource(env, capacity = num_servers)
    
    #amount of personal scanners to adjust
    screening = simpy.Resource(env, capacity = num_scanners)
    
    #set seed for reproducibility
    np.random.seed(seed)
    
    #process customer_run function in the environment
    env.process(customer_run(env, servers, screening))
    #process observe function in the evnironment
    env.process(observe(env, servers, screening))
    
    #empty lists to write data to
    times = []
    obs_times = []
    seq_length = []
    scq_length = []
    
    #run the environment - simulate 12 hours
    env.run(until = 1080)
    
    return times, obs_times, seq_length, scq_length

# Let's try simulating the simple airport for customer arrivals that mimic that of a poisson distribution with lambda = 5. We will simulate for a typical full airport work day (18 Hours)

### SIMULATING WITH 9 SERVERS AND 6 SELF SCANNERS

In [ ]:
times,obs_times,server_q_length,scanner_q_length = airport_model(num_servers = 9, num_scanners=6, seed = 42, lambda_ = 5)

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(times, bins = 10, hist_kws={'density':False}, kde=False)
plt.xlabel('Wait Times')
plt.ylabel('Frequency')
plt.title('Wait Times Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(times)
plt.xlabel('Wait Times')
plt.title('Average Wait Time: {} Minutes'.format(round(np.mean(times),2)))
plt.show()

In [ ]:
obs_times = np.array(obs_times)
obs_times = obs_times / 60

plt.figure(figsize=(15,8))
sns.scatterplot(obs_times, server_q_length)
plt.title('Number of People in Queue')
plt.xlabel('Time of Day')
plt.ylabel('Queue Length')

plt.show()

## CONCLUSION
### MEAN WAIT TIME
After adjusting both the servers and scanners, I eventually arrived at a combination of 9 servers and 6 scanners to keep wait times below 15 minutes. Dropping servers down to 8 increased average wait times to 18 minutes as opposed to the 8 minutes it currently is iwth 9 servers

In [ ]:
np.mean(times)

# BUSY AIRPORT

Let's try the busy airport next

In [ ]:
times,obs_times,server_q_length,scanner_q_length = airport_model(num_servers = 80, num_scanners=62, seed = 42, lambda_ = 50)

In [ ]:
plt.figure(figsize=(15,8))
sns.distplot(times, bins = 10, hist_kws={'density':False}, kde=False)
plt.xlabel('Wait Times')
plt.ylabel('Frequency')
plt.title('Wait Times Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(times)
plt.xlabel('Wait Times')
plt.title('Average Wait Time: {} Minutes'.format(round(np.mean(times),2)))
plt.show()

In [ ]:
obs_times = np.array(obs_times)
obs_times = obs_times / 60

plt.figure(figsize=(15,8))
sns.scatterplot(obs_times, server_q_length)
plt.title('Number of People in Queue')
plt.xlabel('Time of Day')
plt.ylabel('Queue Length')

plt.show()

## CONCLUSION
With 80 servers and 62 self scanners, the average wait time is 11.8 minutes at the very busy airport

In [ ]:
np.mean(times)